In [1]:
import numpy as np
from main import ArduinoReader
from motion_stages import Zaber_2axis_LST1500D as motion_stage
import time
import pandas
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# initialize stage instance and zero axes
stage = motion_stage()
stage.home_axes()

# initialize ArduinoReader
arduino = ArduinoReader('COM7', 9600)  # replace with your port 

In [3]:
x_center = 800.0  # in mm
y_center = 430.0  # in mm
stage.move_x_absolute(x_center)
stage.move_y_absolute(y_center)

In [6]:
calibration_distance = 100.0  # in mm

N_iterations = 10
calibration_results = []

for i in range(N_iterations):
    stage.move_x_absolute(x_center)
    stage.move_y_absolute(y_center)
    time.sleep(5)

    # Record initial sensor readings
    initial_sensor_x, initial_sensor_y = arduino.distanceSinceInit()

    # Move the stage
    stage.move_x_absolute(x_center - calibration_distance)
    stage.move_y_absolute(y_center - calibration_distance)
    time.sleep(5)

    # Record sensor readings after moving
    final_sensor_x, final_sensor_y = arduino.distanceSinceInit()

    # Calculate the sensor's reported movement
    sensor_movement_x = final_sensor_x - initial_sensor_x
    sensor_movement_y = final_sensor_y - initial_sensor_y

    # Calculate calibration factors
    calibration_factor_x = calibration_distance / sensor_movement_x
    calibration_factor_y = calibration_distance / sensor_movement_y

    # Print calibration factors
    print(f"Iteration {i+1} - Calibration factor for X: {calibration_factor_x}")
    print(f"Iteration {i+1} - Calibration factor for Y: {calibration_factor_y}")

    calibration_results.append({
        'Iteration': i+1,
        'Calibration_Factor_X': calibration_factor_x,
        'Calibration_Factor_Y': calibration_factor_y,
        'Sensor_Movement_X': sensor_movement_x,
        'Sensor_Movement_Y': sensor_movement_y,
        'Calibration_Distance': calibration_distance
    })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(calibration_results)

# Append the dataframe to the existing CSV file
with open('calibration_script_results.csv', 'a') as f:
    df.to_csv(f, header=f.tell()==0, index=False)

# Scatter plot of Calibration Factors X and Y over the iterations
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=df, x="Iteration", y="Calibration_Factor_X", color='r')
plt.title('Calibration Factor X over Iterations')

plt.subplot(1, 2, 2)
sns.scatterplot(data=df, x="Iteration", y="Calibration_Factor_Y", color='b')
plt.title('Calibration Factor Y over Iterations')

plt.tight_layout()
plt.show()

# Print out descriptive statistics of the calibration factors
print("\nDescriptive statistics for Calibration Factor X:")
print(df['Calibration_Factor_X'].describe())
print("\nDescriptive statistics for Calibration Factor Y:")
print(df['Calibration_Factor_Y'].describe())

# cleanup
arduino.stop()

Calibration factor for X: -0.022084805653710248
Calibration factor for Y: -0.020004000800160033
Calibration factor for X: -0.022153300841825433
Calibration factor for Y: -0.020300446609825416
Calibration factor for X: -0.02212878955521133
Calibration factor for Y: -0.020292207792207792
Calibration factor for X: -0.021963540522732264
Calibration factor for Y: -0.02087246921310791
Calibration factor for X: -0.021925016443762334
Calibration factor for Y: -0.02012072434607646
Calibration factor for X: -0.02208968411751712
Calibration factor for Y: -0.020181634712411706
Calibration factor for X: -0.022311468094600623
Calibration factor for Y: -0.02
Calibration factor for X: -0.02201188641866608
Calibration factor for Y: -0.020938023450586266
Calibration factor for X: -0.0221483942414175
Calibration factor for Y: -0.019825535289452814
Calibration factor for X: 0.0057670126874279125
Calibration factor for Y: 0.02617115938236064
